# Introduction to MPI - Day 2

In [ ]:
import os
# the jupyter notebook is launched from your $HOME, change the working directory provided a username directory is created under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/mpi/day2"))
os.environ['MPIP'] = "-y -p "

## 1. Collective Communication

In the previous program, the local residuals were calculated on each MPI process and reduced to the root process (the rank 0 process) by reduction operation `MPI_SUM`.
```cpp
double tot_res;
residual  = local_L2_residual(ptr_rows, mesh_size, space, &submesh[0][0], &subrhs[0][0]);
MPI_Reduce(&residual, &tot_res, 1, MPI_DOUBLE, MPI_SUM, 0, world);
```

It is recommended but not required from MPI Standard 4.0 that the same order is maintained for the reduction operation whenever the same arguments are applied to the function. For operations that a specific ordering is significant, one may consider using `MPI_Gather` to collect data to a single process before a local reduction operation.


**`TODO`**: In [lapalce_mpi_collective.c](./laplace_mpi_collective.c), replace the `MPI_Reduce` function with `MPI_Gather` and sum up residuals on the root process. Once you are finished run the next cell. If you are stuck, peek the solution at [soln_laplace_mpi_collective.c](./solution/laplace_mpi_collective.c)

Essential MPI functions needed: `MPI_Gather`

In [ ]:
!make clean && make collective && echo "Compilation Successful!" && mpiexec -np 12 ./laplace_mpi_collective 300 1000 Jacobi > convergence_collective.txt
!tail -20 convergence_collective.txt 

## 2. One-sided Communication

The mechanism of one-sided communication (aka remote memory access) proposes an entirely different comunication pattern than the message-passing examples that we have seen. There are no senders and receivers involved in this pattern, meaning, messages do not need to match before the transfer. Instead, by creating a memory object called window at the target in which the memory window is accessed, the origin, in which the call is performed, can put data to and get data from that window followed by whatever local load/store operations.

In our example [laplace_mpi_win.c](./laplace_mpi_win.c), each MPI process is used as both the origin and the target - when the window is created and exposed, it is treated as the target, when an RMA call is performed, it is the origin. 

`As the target`, two window objects are created with separate allocated memory in an MPI process, one for the top row of ghost nodes, the other for the bottom row of ghost nodes.
```cpp
MPI_Win upper_win, lower_win;
double *upper_bnd, *lower_bnd;
/* win for the top row of ghost nodes */
MPI_Alloc_mem(sizeof(double )*mesh_size, MPI_INFO_NULL, &upper_bnd);
MPI_Win_create(upper_bnd, mesh_size *sizeof(double), sizeof(double),\
               MPI_INFO_NULL, MPI_COMM_WORLD, &upper_win);

/* win for the bottom row of ghost nodes */
MPI_Alloc_mem(sizeof(double )*mesh_size, MPI_INFO_NULL, &lower_bnd);
MPI_Win_create(lower_bnd, mesh_size *sizeof(double), sizeof(double),\
               MPI_INFO_NULL, MPI_COMM_WORLD, &lower_win); 
```

`As the origin`, the MPI process is very much like the sender doing `MPI_Send`, it needs to transfer out the top and bottom full nodes for the neighbouring processes to update their ghost nodes, however, this is done by using `MPI_Put`.

```cpp
/* put the bottom row of full nodes to the rank below */
MPI_Put(submesh[1], mesh_size, MPI_DOUBLE, lower, 0, mesh_size, MPI_DOUBLE, upper_win);

/* put the top row of full nodes to the rank above */
MPI_Put(submesh[*ptr_rows - 2], mesh_size, MPI_DOUBLE, upper, 0, mesh_size, MPI_DOUBLE, lower_win);   
```

Next, after completing the RMA operation the same MPI process needs to load/store data from its window object to update its own ghost nodes. This makes it the `target`.

```cpp
/* update the full bottom and top of submesh */
if (rank > 0 && rank < (cells -1)){
    for (int i=0; i< mesh_size; i++){
        submesh[0][i] = lower_bnd[i]; /* bottom */
        submesh[*ptr_rows-1][i] = upper_bnd[i]; /* top */            
}
}
else if (rank ==0 ){ /* bottom rank only updates its top full row */
    for (int i = 0; i< mesh_size; i++ ){

        submesh[*ptr_rows-1][i] = upper_bnd[i];
    }
}

else if (rank == (cells -1) ){ /* top rank only updates its bottom full row */ 
    for (int i = 0; i <mesh_size; i++){
        submesh[0][i] = lower_bnd[i];
    }
}
```

Notice that the transition for this MPI proceess 
$$Origin \to Target \to Origin$$
where each phase is concluded by a sychronisation call `MPI_Win_fence`.

Again, the window object doesn't have to match with a RMA operation `MPI_Put`. In contrast, it is designed to be exposed by multiple meessages from any process in the communicator group.

**`TODO`**: Replace the two window objects in [laplace_mpi_win.c](./laplace_mpi_win.c) with a single window object. Use this window object to host both top and bottom ghost nodes for the neighbouring processes' access. Once you are finished run the next cell. If you are stuck, peek the solution at [soln_laplace_mpi_win.c](./solution/laplace_mpi_win.c)

Essential MPI functions needed: `MPI_Win_create`, `MPI_Alloc_mem`, `MPI_Win_fence`, `MPI_Put`, `MPI_Win_free`

In [ ]:
!make clean && make win && echo "Compilation Successful!" && mpiexec -np 12 ./laplace_mpi_win 300 1000 Jacobi > convergence_win.txt
!tail -20 convergence_win.txt 

## 3. MPI-IO

After the iterative method is terminated with an acceptable magnitude of the error, we want to write the approximation over the unit square domain for the visualisation, validation and whatnots. The question now is the following: each MPI process only possesses a subdomain so how to merge the data to a common file?

Traditionally, one may either send all the subdomains to a root process, such is called master-slave communication mode, or output each individual subdomains for a different program to scan and combine.

MPI-IO provides the ability for multiple MPI processes to write into a common file. The basic MPI-IO procedures resembles with the POSIX IO:
```cpp
/* Open a file */
MPI_File_open(MPI_COMM_WORLD, file_name, MPI_MODE_CREATE|MPI_MODE_WRONLY, MPI_INFO_NULL, &fp);

/* Write to a file */
MPI_File_write_at(fp, offset, &submesh[1][0], count, MPI_DOUBLE, &IO_status );

/* Close a file */
MPI_File_close(&fp);

```
Each process starts to write at the offset position of the common file. It is the programmer's responsibility to ensure the offset is set correctly.

**`TODO`**: Complete the output routine for middle rank MPI processes in [laplace_mpi_io.c](./laplace_mpi_io.c). Once you are finished run the next cell. If you are stuck, peek the solution at [soln_laplace_mpi_io.c](./solution/laplace_mpi_io.c)

In [ ]:
!make clean && make io && echo "Compilation Successful!" && mpiexec -np 8 ./laplace_mpi_io 300 1000 Jacobi > convergence_io.txt
!tail -20 convergence_io.txt 



Inspect the output data file [laplace-soln-whole](./laplace-soln-whole).

Yes, MPI-IO only supports unformatted binary files, so you will need to interpret the unformatted data by whichever tools you like. (MATLAB is my go-to for this task.)

## 4. Profile with mpiP
There are many profilers available for profiling MPI codes. We use mpiP in this workshop in favour of its simplicity, light weight and portability as it is built on PMPI instrumentation. There are more comprehensive profiling tools for tracing the fine-grained events.

In this section, we profile our previous programs with different communication models.

### 4.1 Profile blocking communication

In [ ]:
!cd ../day1/solution &&  make clean && make blockingP && echo "Compilation Successful!"  && mpiexec -np 24 ./laplace_mpiP_blocking 1000 30 Jacobi > /dev/null
!cd ../day1/solution && cat *.mpiP
!cd ../day1/solution && rm -r *.mpiP

### 4.2 Profile nonblocking communication

In [ ]:
!cd ../day1 &&  make clean && make nonblockingP && echo "Compilation Successful!"  && mpiexec -np 24 ./laplace_mpiP_nonblocking 1000 20 Jacobi > nonblocking.txt
!cd ../day1 && cat *.mpiP 
!cd ../day1/solution && rm -r *.mpiP

### 4.3 Profile persistent communication

In [ ]:
!cd ../day1 && make clean && make persistentP && echo "Compilation Successful!"  && mpiexec -np 24 ./laplace_mpiP_persistent 1000 30 Jacobi > /dev/null
!cd ../day1 && cat *.mpiP 
!cd ../day1 && rm -r *.mpiP

### 4.3 Profile one-sided communication

In [ ]:
!cd ../day2  &&  make clean && make winP && echo "Compilation Successful!"  && mpiexec -np 24 ./laplace_mpiP_win 1000 30 Jacobi > /dev/null
!cd ../day2 && cat *.mpiP
!cd ../day2 && rm -r *.mpiP  